In [2]:
import xrayutilities as xu
import numpy as np
from matplotlib.pylab import plt
import pandas as pd

import lmfit
%matplotlib notebook


In [3]:
def TetragonalElasticTensor(c11, c12, c13, c33, c44, c66):
    """
    Assemble the 6x6 matrix of elastic constants for a cubic material from the
    three independent components of a cubic crystal
    Parameters
    ----------
    c11, c12, c44 : float
        independent components of the elastic tensor of cubic materials
    Returns
    -------
   cij :    ndarray
        6x6 matrix with elastic constants
    """
    m = np.zeros((6, 6), dtype=np.double)
    m[0, 0] = c11
    m[1, 1] = c11
    m[2, 2] = c33
    m[3, 3] = c44
    m[4, 4] = c44
    m[5, 5] = c66
    m[0, 1] = c12
    m[0, 2] = m[1, 2] = c13
    m[1, 0] = c12
    m[2, 0] = m[2, 1] = c13

    return m

In [4]:
#Define graphic parameters
plt.rcParams['font.size'] = 16.0

#Define beam energy
en = 8010
#Define Bragg-Brentano peak
H, K, L = (0, 0, 2)
#Define θ resolution
resolai=0.01

In [7]:
#Range of analysis
start = 102
final = 924

In [5]:

#-----------------------------------------------------------------------------
#Fit function -
#Contribuiton of Prof. Dr. Christoph Friedrich Deneke in classes
#F838 - Métodos de Física Experimental VII - 1S/2020
#------------------------------------------------------------------------------

def fit_model_residual(pars,ai,**kwargs):
    '''
    Model function to calculate the difference between experimental data and model
    
    As the standart model of xrayutilities does not support changing concentration
    of a pseudomorphic stack, we have to build the stack and than put it back into
    the model.
    
    Parameters
    ----------
    pars : LMFit paramters
        Paramters to used to calc the model and to be refined by minimizer
    ai : Numpy array
        Incident angle of the beam - in case of a symetrical reflection, this 
        theta of the Theta/2Theta scan
    **kwargs : data and standart devison of the data - numpy arryas
        Experimental data and its standart deviation (eps). If eps not
        given, we will weight the data by 1/data.

    Returns
    -------
    TYPE
        Either the model values (if no data given) or the weighted difference 
        between experiment and model

    '''
    #Paramters given to the function
    data = kwargs.get('data', None)
    eps = kwargs.get('eps', None)
    pvals = pars.valuesdict()
   
    #Experimental paramteters
    #We use the wavelength to define the energy
    
    en = xu.utilities.lam2en(pvals.get('wavelength'))
    H, K, L = (0, 0, 2)
    resolai=0.001
    
    #Define Unit Cells
    STO = xu.materials.Crystal('SrTiO', xu.materials.SGLattice(
    123, 3.9050, 3.9050, atoms=('Sr', 'Ti', 'O', 'O'), pos=('1a', '1d', '1c', '2e'))
                           , xu.materials.CubicElasticTensor(38.9e+10, 10.5e+10, 15.5e+10))

    
    LSMO = xu.materials.Crystal('LaSrMnO', xu.materials.SGLattice(
    123, pvals.get('LSMO_a'), pvals.get('LSMO_c'), atoms=('La', 'Sr', 'Mn', 'O', 'O'), pos=('1a', '1a', '1d', '1c', '2e'),
    occ=(2/3, 1/3, 1, 1, 1)), TetragonalElasticTensor(pvals.get('Tensor_c11'), pvals.get('Tensor_c12'),
                                                                  pvals.get('Tensor_c13'), pvals.get('Tensor_c33'),
                                                                  pvals.get('Tensor_c44'), pvals.get('Tensor_c66')))

    
    sub=xu.simpack.Layer(STO, np.inf)
    lay1=xu.simpack.Layer(LSMO , pvals.get('LSMO_thickness'), relaxation=pvals.get('LSMO_relaxation'))
    sequ1=sub + lay1
    pls = xu.simpack.PseudomorphicStack001('pseudo', sequ1)
  

    md = xu.simpack.DynamicalModel(pls, energy=en, resolution_width=resolai)
      
    #Calculate the Model, scale it by multiplying a scale factor and add a background
    #
    #Scaling and background are by now implmented into the xrayutility model.
    #
    model = md.simulate(ai-pvals.get('shift'), hkl=(H, K, L))*pvals.get('scale')+pvals.get('bg_const')
    
    #Calculate the value for the minimizer
    #
    #For better conversion and stability of the fit, difference is weigthed by 1/data
    #Make sure to grap all NaN or Inf values before returning the result!
    
    if data is None:
        return model
    if eps is None:
        weight = np.reciprocal(data)
        weight[np.isinf(weight)] = 1
        return (model - data)*weight
    return (model - data)/eps


In [6]:
#-----------------------------------------------------------------------------
#Residual cb function to see, whats going on -
#Contribuiton of Prof. Dr. Christoph Friedrich Deneke in classes
#F838 - Métodos de Física Experimental VII - 1S/2020
#------------------------------------------------------------------------------

def cb_func(params, niter, resid, ai, **kwargs):
    '''
    
    Parameters
    ----------
    params : LMFit paramters
        Current LMFit paramters during refinment
    niter : Integer
        Loop of refinment (can be limited by maxfev).
    resid : Numpy array
        Return of the model function (normaly (data-model))
    ai : Numpy array
        Theta value incase of symetrical relfection
    **kwargs : TYPE
        Same as

    Returns
    -------
    None.

    '''
    print('Iteration:{:04d} Chi^2: {:9.3e} Reduced Chi^2: {:2.3e}'.format(niter, np.sum(resid**2),1/len(resid)*np.sum(resid**2)))
    
    data = kwargs.get('data', None)
    eps = kwargs.get('eps', None)
    
    if eps is None:
            weight = 1/data
            difference=resid/weight
    else:
       weight = 1/eps
       difference=resid*(1/eps)
       
    weight[np.isinf(weight)] = 1
    
    #Define the Rietveld matrix
    
    M = np.sum((difference)**2 * weight)
    Rp = np.sum(np.abs(difference))/np.sum(data)
    Rwp = np.sqrt(M / np.sum(weight * data**2))
    chi2 = M / len(data)
    Rwpexp = Rwp / np.sqrt(chi2)
    
    #print(weight)
    print('Rp=%.4f Rwp=%.4f Rwpexp=%.4f chi2=%.4f'
              % (Rp, Rwp, Rwpexp, chi2))#, len(data)))



In [8]:
#-----------------------------------------------------------------------------
#Doing the fit
#------------------------------------------------------------------------------

#Define some names for convinience
sname = 'Sample Name'
fname='filename'
x_data='Theta'
y_data='Intensity'
adata = pd.read_csv(fname, sep = ' ')
adata

In [15]:
exp = pd.DataFrame(columns={'Theta','Intensity'})
exp['Theta'] = adata['Theta'].iloc[start:final]
exp['Intensity'] = adata['Roi3'].iloc[start:final]
#Define the fitting data
edata=np.array(exp['Intensity'])
theta=np.array(exp['Theta'])

In [10]:


#Define parameters as used in fit function
#
#You will have to adapt the paramters to fit your layer model
#

p = lmfit.Parameters()
#          (Name                   ,Value        ,  Vary ,   Min  ,  Max  , Expr)
p.add_many(('wavelength'           ,1.545937636324193           , False  ,1.53  ,1.55  , None),
           ('scale'                ,1071100000             , False ,5e6    ,5e11  , None),
           ('shift'                ,0.000                       , False ,0      ,0.05  , None),
           ('bg_const'             ,142.108                           , False,0      ,1e3   , None),
           ('LSMO_thickness'       ,60.444                      , False ,0     ,200    , None),
           ('LSMO_c'               ,3.851                       , False ,3.7    ,4     , None),
           ('LSMO_a'               ,3.909                        , False ,3.7    ,4     , None),
           ('LSMO_relaxation'      ,0                           , False ,0    ,1     , None),
           ('Tensor_c11'           ,2.272e+9                   , False ,2.272e10      ,2.272e17     , None),
           ('Tensor_c12'           ,1.586e+11                   , False ,1.586e10      ,1.586e17     , None),
           ('Tensor_c13'           ,1.586e+11                   , False ,1.586e10      ,1.586e17  , None),
           ('Tensor_c33'           ,2.272e+9                   , False ,2.272e10      ,2.272e17  , None),
           ('Tensor_c44'           ,6.65e+10                    , False ,6.65e9      ,6.65e17  , None),
           ('Tensor_c66'           ,6.65e+10                    , False ,3.4300e+9     ,3.4300e+12     , None)
            )
           
#Define x and y data - x-data has to be a numpy-array. Otherwise diffraction models do not work
#

p

In [120]:
#Define the fitting data
edata=np.array(exp['Intensity'])
theta=np.array(exp['Theta'])

#Book keeping by passing the used x array and a inital simulation back to the DataFrame
#exp[x_data]=theta
exp['Simulation']=fit_model_residual(p,theta)

exp.to_csv('edata.csv')
#theta

In [121]:
#Do the fit and get output
minimizer = lmfit.Minimizer(fit_model_residual, p, fcn_args=([theta]),
                            fcn_kws={'data':edata, 'eps':edata}, iter_cb=cb_func)

#For this kind of fit, sometimes is worth trying a different minimizer - e.g. the ampo
#is an interesting choice, if you have already some idea of the parameters

In [11]:
#res=minimizer.minimize(method='ampgo')
res=minimizer.minimize(maxfev=2000)
#res=minimizer.minimize(method='emcee3')
lmfit.report_fit(res)

In [12]:
#Book keeping and make a simulation with the found paramters
exp['Fit']=fit_model_residual(res.params,theta)

#Prepare a string for report
p1=p.valuesdict()
result=res.params.valuesdict()

output=''


for key in result:
    output+=f'{key}: {result[key]:.3f}\n'# ({p1[key]:.3f})\n'

print(output)

In [13]:
#Let's plot the stuff....
fig = plt.figure()
fig.subplotpars.hspace=0

spec1=plt.GridSpec(2,1,width_ratios=[1], height_ratios=[4, 1])
spec1.update(hspace=0.0)

ax1=fig.add_subplot(spec1[0])
ax1.text(.05,.95, output,fontsize=9,horizontalalignment='left',
         verticalalignment='top', transform=ax1.transAxes)
ax1.semilogy(exp[x_data],exp[y_data], '+', color = 'blue', label = sname)
ax1.semilogy(exp[x_data],exp['Fit'], label='Simulação', color = 'red')
ax1.semilogy(exp[x_data],exp['Simulation'], label='Fit', color ='green')
ax1.tick_params(direction='in', which='both',top=True,right=True,
                labelbottom=False)
ax1.legend(fontsize='small')
#ax1.set_xlim(0, 100) 
ax1.set_ylabel('Intensidade (u. a.)')
plt.title('Padrão de difração de raios X e simulação para ' + sname, fontsize= 12, weight = 'bold')

fig.subplotpars.hspace=0

ax2=fig.add_subplot(spec1[1])
ax2.plot(exp[x_data],np.abs(res.residual), color= 'blue')
ax2.tick_params(direction='in', which='both',top=True,right=True,labelbottom=True)
ax2.set_xlabel('Theta [°]')
ax2.set_ylabel('Resíduo')
#ax2.set_xlim(18, 28) 
#plt.tight_layout()
#plt.show()


plt.tight_layout() # ajusta para salvar
plt.savefig(sname + '.pdf',bbox_to_anchor=True)
plt.show()

#And potentially save everything back

exp=exp.drop(['Unnamed: 0'],axis=1)
exp.to_csv(fname, index=False)
res.params.dump('fit_param_' + sname.replace (" ", "_") + '.json')